# SUMMARY

In [740]:
from sklearn import preprocessing 
from sklearn.model_selection import GridSearchCV 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import RandomForestRegressor

#Import some extention libraries 
%matplotlib inline 
import warnings
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np
import pandas as pd 
import os
import glob
import xlrd
plt.style.use('ggplot')
pd.options.mode.chained_assignment = None

In [743]:
path = r'/Users/dong/Desktop/lwh'
dirlist = []
for dirpath,dirname,filename in os.walk(path):
    for i in filename:
        dirlist.append(os.path.join(dirpath,i))

In [744]:
dirlist

['/Users/dong/Desktop/lwh/042018LWH.xlsx',
 '/Users/dong/Desktop/lwh/042017.xlsx',
 '/Users/dong/Desktop/lwh/032018LWH.xlsx',
 '/Users/dong/Desktop/lwh/022018.xlsx']

In [745]:
dflist = []
for i in dirlist:
    dflist.append(pd.read_excel(i))

In [746]:
mydf = pd.concat(dflist)

In [747]:
df = pd.ExcelWriter(r'mydf.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})

In [748]:
mydf.to_excel(df,index=None)

In [749]:
pd.set_option('display.max_columns', None)
#mydf

In [750]:
df = mydf.sort_values("訂單日期", ascending = True) #ascending=False 將序
df.reset_index(inplace=True, drop=True) # rearrange index number 

In [751]:
dPick_up = df
dPick_up['訂單日期'] = pd.to_datetime(dPick_up.訂單日期,format='%Y-%m') # 提取出時間信息
dPick_up['month'] = dPick_up.訂單日期.values.astype('datetime64[M]')

# Filter 

In [752]:
df_filter = df
df_filter = df_filter.loc[df_filter["訂單狀態"] != "已取消"]
#df_filter = df_filter.loc[df_filter["送貨狀態"] != "已退貨"]
#df_filter = df_filter.loc[df_filter["送貨狀態"] != "退貨中"]
#df_filter = df_filter.loc[df_filter["串接物流貨態"] != "未執行"]
#df_filter = df_filter.loc[df_filter["付款狀態"] != "未付款"]

# Business Health

In [753]:
dB = df_filter
dB['訂單日期'] = pd.to_datetime(dB.訂單日期,format='%Y-%m') # 提取出时间信息
dB['month'] = dB.訂單日期.values.astype('datetime64[M]')
dtest = dB
dtest["month"] = dtest["month"].astype(str)
Revenue = dtest.groupby('month').訂單合計.agg(['sum','count', 'mean'])
Revenue = Revenue.rename(columns = {'sum':'Revenue',"count":"Order","mean":"AOV"})
Revenue["AOV"] = Revenue.AOV.round(decimals = 0)
Revenue["Revenue"] = Revenue.Revenue.astype(str)
Products_sold = dtest.groupby('month').數量.agg(['sum'])
Revenue["AOV"] = Revenue["AOV"].astype(int)
Products_sold = Products_sold.rename(columns = {'sum':'Products sold'})
Business_Health = Revenue.join(Products_sold)
Business_Health['Products per order'] = Business_Health['Products sold']/Business_Health['Order']
Business_Health['Products per order']  = Business_Health['Products per order'] .round(decimals = 1)
#Revenue["month"] = Revenue["month"].astype(str)
#Business_Health = Business_Health.T
Business_Health = Business_Health.T
#row_month = Business_Health.month.sort_values().astype('str').unique()
#Business_Health.rows = row_month
Business_Health 


month,2017-04-01,2018-02-01,2018-03-01,2018-04-01
Revenue,1995509.0,4486592.0,5506572.0,4850942.0
Order,1556,3848,4584,4186
AOV,1282,1166,1201,1159
Products sold,2256,8086,9534,8915
Products per order,1.4,2.1,2.1,2.1


# Customer Profile

In [754]:
Customer_amount = dtest.pivot_table(index='電郵',columns='month',values='訂單合計',aggfunc="count").fillna(0)
columns_month = dtest.month.sort_values().astype('str').unique()
Customer_amount.columns = columns_month
#Customer_amount.loc['customers']= Customer_amount.count()
#Customer_amount.loc['Total Customer']= Pick_up_amount.apply(lambda x if x == 1 x.sum)
#Customer_amount.applymap(lambda x: 1 if x > 1 else np.NaN)
#Customer_amount.loc['Total Customer'] = Customer_amount.count()#applymap(lambda x: 1 if x > 1 else np.NaN)
Customer_amount = Customer_amount.apply(lambda x : pd.value_counts(x))
Customer_amount = Customer_amount.iloc[1:]#iloc 切片去掉0
Customer_amount.loc['Customers']= Customer_amount.sum()
Customer_amount.loc['Customers']= Customer_amount.loc['Customers'].astype(int)
Customer_amount.loc['% of repeat orders'] = Customer_amount.iloc[0:].sum()
Customer_amount.loc['customers bought once']= Customer_amount.iloc[0]
Customer_amount.loc['customers bought more than once'] = Customer_amount.loc['Customers']-Customer_amount.loc['customers bought once']
Customer_amount.loc['% of repeat orders'] = Customer_amount.loc['customers bought more than once']/Customer_amount.loc['Customers']
Customer_amount.loc['% of repeat orders']= Customer_amount.loc['% of repeat orders']*100
Customer_amount.loc['% of repeat orders'] = Customer_amount.loc['% of repeat orders'].round(decimals = 1)
#Customer_amount.loc['% of repeat orders'] = Customer_amount.loc['% of repeat orders']+"%"
Customer_amount = Customer_amount.iloc[-4:]
Customer_amount.loc['Customers'] = Customer_amount.loc['Customers'].astype(str)
Customer_amount.loc['customers bought once'] = Customer_amount.loc['customers bought once'].astype(str)
Customer_amount.loc['customers bought more than once'] = Customer_amount.loc['customers bought more than once'].astype(str)
Customer_amount.loc['% of repeat orders'] = Customer_amount.loc['% of repeat orders'].astype(str)
Customer_amount.loc['% of repeat orders'] = Customer_amount.loc['% of repeat orders']+"%"
Customer_amount


,2017-04-01,2018-02-01,2018-03-01,2018-04-01
Customers,1517.0,3727.0,4420.0,4010.0
% of repeat orders,2.3%,3.0%,3.4%,4.0%
customers bought once,1482.0,3616.0,4271.0,3851.0
customers bought more than once,35.0,111.0,149.0,159.0


# Pick up Analysis

In [755]:
dPick_up
dPick_up = dPick_up.loc[dPick_up["付款方式"] == "7-11取貨付款"]
dPick_up = dPick_up.rename(columns = {'送貨狀態':'Pick up Analysis'})
Pick_up_amount = dPick_up.pivot_table(index='Pick up Analysis',columns='month',values='訂單合計',aggfunc="count").fillna(0)
columns_month = dPick_up.month.sort_values().astype('str').unique()
Pick_up_amount.columns = columns_month
Pick_up_amount#.sort_values("month", ascending = True)
Pick_up_amount.loc['Total Pick up count']= Pick_up_amount.sum()
Pick_up_amount.loc['No pick up count'] = Pick_up_amount.loc['已退貨']+ Pick_up_amount.loc['退貨中']
Pick_up_amount.loc['% of no pick up'] = Pick_up_amount.loc['No pick up count']/Pick_up_amount.loc['Total Pick up count']
Pick_up_amount.loc['% of no pick up'] = Pick_up_amount.loc['% of no pick up'].round(decimals = 3)
Pick_up_amount.loc['% of no pick up'] = Pick_up_amount.loc['% of no pick up']*100
#Pick_up_amount.loc['% of no pick up'] = Pick_up_amount.loc['% of no pick up'].astype(str)
#Pick_up_amount.loc['% of no pick up'] = Pick_up_amount.loc['% of no pick up']+"%"
Pick_up_amount.loc['Total Pick up count'] = Pick_up_amount.loc['Total Pick up count'].astype(str)
Pick_up_amount.loc['No pick up count'] = Pick_up_amount.loc['No pick up count'].astype(str)
Pick_up_amount.loc['% of no pick up'] = Pick_up_amount.loc['% of no pick up'].astype(str)
Pick_up_amount.loc['% of no pick up'] = Pick_up_amount.loc['% of no pick up']+"%"
Pick_up_amount = Pick_up_amount[-3:]
Pick_up_amount

,2017-04-01,2018-02-01,2018-03-01,2018-04-01
Pick up Analysis,,,,
Total Pick up count,1532.0,3401.0,4061.0,3593.0
No pick up count,106.0,207.0,222.0,176.0
% of no pick up,6.9%,6.1%,5.5%,4.9%


In [756]:
dpick = df
dpick['訂單日期'] = pd.to_datetime(dpick.訂單日期,format='%Y-%m') # 提取出时间信息
dpick['month'] = dpick.訂單日期.values.astype('datetime64[M]')
dpick = dpick.dropna(subset = ["訂單合計"])
dpick['超取取貨時間']=pd.to_datetime(dpick['超取取貨時間']) 
dpick['超取到達時間']=pd.to_datetime(dpick['超取到達時間']) 
#['超取到達時間'] = pd.to_datetime(dy.超取到達時間,format='%Y-%m-%d') # 提取出时间信息
#dy['arrival'] = dy.超取到達時間.values.astype('datetime64[D]')
#dy['超取取貨時間'] = pd.to_datetime(dy.超取取貨時間,format='%Y-%m-%d') # 提取出时间信息
#dy['pickup'] = dy.超取取貨時間.values.astype('datetime64[D]')
dpick['pickupday'] = dpick['超取取貨時間']-dpick['超取到達時間']
dpick = dpick.dropna(subset = ["pickupday"])
dpick.pickupday = dpick.pickupday.astype(str)
split = pd.DataFrame((x.split('d') for x in dpick['pickupday']),index=dpick.index,columns=['day','min'])
dpick = pd.merge(dpick,split,right_index=True, left_index=True)
dpick["month"] = dpick["month"].astype(str)
dpick["day"] = dpick["day"].astype(int)
days = dpick.groupby('month').day.agg(['mean'])
days = days.rename(columns = {'mean':'Avg pick up day'})
Avg_day = days.T

In [757]:
Pick_up_overview = pd.concat([Pick_up_amount,Av_day]) 

In [758]:
pd.concat([Business_Health,Customer_amount,Pick_up_overview]) 

month,2017-04-01,2018-02-01,2018-03-01,2018-04-01
Revenue,1995509.0,4486592.0,5506572.0,4850942.0
Order,1556,3848,4584,4186
AOV,1282,1166,1201,1159
Products sold,2256,8086,9534,8915
Products per order,1.4,2.1,2.1,2.1
Customers,1517.0,3727.0,4420.0,4010.0
% of repeat orders,2.3%,3.0%,3.4%,4.0%
customers bought once,1482.0,3616.0,4271.0,3851.0
customers bought more than once,35.0,111.0,149.0,159.0
Total Pick up count,1532.0,3401.0,4061.0,3593.0


# Payment Overview

In [764]:
dtest = dB
dpayment = dtest.dropna(subset = ["訂單合計"])
dpayment = dpayment.rename(columns = {'付款方式':'Payment'})
Payment_amount = dpayment.pivot_table(index='Payment',columns='month',values='訂單合計',aggfunc="count").fillna(0)
columns_month = dpayment.month.sort_values().astype('str').unique()
Payment_amount.columns = columns_month
#Payment_amount_percentage = Payment_amount.apply(lambda x : x/x.sum())
#Payment_amount.loc["% of Payment"] = ""
Payment_amount
#Payment_amount_percentage = Payment_amount.apply(lambda x : x/x.sum())
#pd.concat([Payment_amount,Payment_amount_percentage])  #上下合併


#pd.merge(Payment_amount,Payment_amount_percentage) 
#Payment_amount = Payment_amount.merge(Payment_amount_percentage)

,2017-04-01,2018-02-01,2018-03-01,2018-04-01
Payment,,,,
7-11取貨付款,1380.0,3291.0,3964.0,3518.0
ATM櫃員機（經由綠界),9.0,0.0,0.0,0.0
信用卡付款,0.0,461.0,527.0,560.0
信用卡（經由綠界),130.0,0.0,0.0,0.0
宅配貨到付款,37.0,0.0,0.0,0.0
特殊付款方式,0.0,1.0,0.0,0.0
銀行轉帳／ATM,0.0,95.0,93.0,108.0


# Delivery Overview

In [765]:
dtest = dB
delivery = dtest.dropna(subset = ["訂單合計"])
delivery = delivery.rename(columns = {'送貨方式':'Delivery'})
Delivery_amount = delivery.pivot_table(index='Delivery',columns='month',values='訂單合計',aggfunc="count").fillna(0)
columns_month = delivery.month.sort_values().astype('str').unique()
Delivery_amount.columns = columns_month
#Delivery_amount = Delivery_amount.nlargest(5,"2018-03-01")
Delivery_amount

,2017-04-01,2018-02-01,2018-03-01,2018-04-01
Delivery,,,,
$180_宅配寄送,45.0,8.0,0.0,0.0
$180_宅配寄送_2月開始靠近年節，宅配將無法指定時間到貨喔!,0.0,195.0,0.0,0.0
$200_冷凍宅配寄送,0.0,0.0,1.0,0.0
$200_宅配寄送,0.0,17.0,177.0,140.0
$80_7-11 B2C 取貨不付款 ( 使用信用卡刷卡/銀行轉帳 ),131.0,337.0,440.0,527.0
$80_7-11取貨付款(若需使用信用卡請選擇下一個選項),1380.0,3291.0,3964.0,3518.0
West Malaysia delivery,0.0,0.0,1.0,0.0
港澳順豐寄送_貨到付寄送運費(一公斤27港幣),0.0,0.0,1.0,1.0


In [766]:
#Delivery_amount.apply(lambda x : x/x.sum())

# Products Overview

In [762]:
Products_amount = dtest.pivot_table(index='商品名稱',columns='month',values="商品特價",aggfunc="sum").fillna(0)
columns_month = dtest.month.sort_values().astype('str').unique()
Products_amount.columns = columns_month
Products_amount.nlargest(10,"2018-03-01")

,2017-04-01,2018-02-01,2018-03-01,2018-04-01
商品名稱,,,,
綠意盎然純棉床包組 (植物),0.0,119896.0,242032.0,94000.0
刺子繡水洗棉床包組,0.0,79426.0,185866.0,83854.0
英式格紋・ 床包組 (全藍),30024.0,182048.0,140110.0,77678.0
地質系・掛毯,0.0,144380.0,126712.0,49460.0
童話化妝品收納包,0.0,24875.0,104558.0,26715.0
迷走叢林純棉床包組,0.0,0.0,96012.0,85742.0
小清新水洗棉・涼被,0.0,0.0,83600.0,149650.0
沐浴晨光純棉床包組,0.0,0.0,81654.0,22034.0
細條紋天竺棉床包組,0.0,61454.0,80854.0,45756.0


In [763]:
#Products_amount = dtest.groupby('m').數量.agg(['sum'])